In [83]:
# import 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import pickle
import torch
import torch.nn as nn
import yaml
import torchvision
from yolov5.models.yolo import Model
from yolov5.utils.general import intersect_dicts
from yolov5.utils.general import non_max_suppression

### Load Models

In [251]:
def get_model(hyp, weights, cfg):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if isinstance(hyp, str):
        with open(hyp, errors='ignore') as f:
            hyp = yaml.safe_load(f)  # load hyps dict


    pretrained = weights.endswith('.pt')
    if pretrained:
        ckpt = torch.load(weights, map_location='cpu')  # load checkpoint to CPU to avoid CUDA memory leak
        model = Model(cfg or ckpt['model'].yaml, ch=3, nc=2, anchors=hyp.get('anchors')).to(device)  # create
        exclude = ['anchor']
        csd = ckpt['model'].float().state_dict()  # checkpoint state_dict as FP32
        csd = intersect_dicts(csd, model.state_dict(), exclude=exclude)  # intersect
        model.load_state_dict(csd, strict=False)  # load

    model.hyp = hyp  # attach hyperparameters to model
    model.eval()

    return model

hyps = "./yolov5/data/hyps/hyp.scratch-low.yaml"
rm_weights = "../Weights/yolov5/rider_motor_cl/best.pt"
hnh_weights = "../Weights/yolov5/h_nh_run/best.pt"
cfg = "./yolov5/models/yolov5m.yaml"
rider_motor_model = get_model(hyps, rm_weights, cfg)
helmet_no_helmet_model = get_model(hyps, hnh_weights, cfg)

Overriding model.yaml nc=80 with nc=2

                 from  n    params  module                                  arguments                     
  0                -1  1      5280  models.common.Conv                      [3, 48, 6, 2, 2]              
  1                -1  1     41664  models.common.Conv                      [48, 96, 3, 2]                
  2                -1  2     65280  models.common.C3                        [96, 96, 2]                   
  3                -1  1    166272  models.common.Conv                      [96, 192, 3, 2]               
  4                -1  4    444672  models.common.C3                        [192, 192, 4]                 
  5                -1  1    664320  models.common.Conv                      [192, 384, 3, 2]              
  6                -1  6   2512896  models.common.C3                        [384, 384, 6]                 
  7                -1  1   2655744  models.common.Conv                      [384, 768, 3, 2]             

In [252]:
# infer_RHNH = keras.models.load_model('../../Weights/helmet_no_helmet_512')
infer_RHNH = helmet_no_helmet_model
infer_rider_motor = rider_motor_model


### Load Images from Validation Set

In [253]:
import os

frames = []
frames2 = []

# Storing all the Validation Images 
# Images are all files with .jpg in data/validation_data_234Images

for file in os.listdir('../data/testing_data_310Images/final_test_set/'):
    if file.endswith('.jpg') or file.endswith('.jpeg'):
        # read the image and storing 2 copies of it for annotation purposes 
        img1 = cv2.imread(os.path.join('../data/testing_data_310Images/final_test_set/', file))
        img2 = img1.copy()
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
        frames.append(img1)
        frames2.append(img2)

In [254]:
# limit to first 10 images
frames = frames[0:4]
frames2 = frames2[0:4]

print("No of Images", len(frames))
# Print 3 random images in a single row
fig, ax = plt.subplots(1, 3, figsize=(20, 20))
print('\t\t\tRandom Images from Test Data')
random_indices = np.random.randint(0, len(frames), 3)
for i, idx in enumerate(random_indices):
    ax[i].imshow(frames[idx])
    ax[i].axis('off')
plt.show()

No of Images 4
			Random Images from Test Data


### Prepare Image input for Model

In [255]:
input_size = 640
all_batch_data = []
for i, frame in enumerate(frames):
    frame_size = frame.shape[:2]
    image_data = cv2.resize(frame, (input_size, input_size))
    image_data = image_data / 255.
    image_data = image_data[np.newaxis, ...].astype(np.float32)
    image_data = torch.from_numpy(image_data)
    image_data = image_data.permute((0,3,1,2))
    all_batch_data.append(image_data)

### R + M Model Prediction

In [256]:
all_rider_motor_bbox = []

for batch_data in all_batch_data:
    # rider_motor_bbox = infer_rider_motor.predict(batch_data)
    rider_motor_bbox = infer_rider_motor(batch_data)
    all_rider_motor_bbox.append(rider_motor_bbox)

print(all_rider_motor_bbox)

[(tensor([[[8.83328e+00, 5.35988e+00, 9.29722e+00,  ..., 3.45721e-05, 5.62062e-01, 5.74551e-01],
         [1.24181e+01, 5.52407e+00, 1.43080e+01,  ..., 3.61809e-05, 5.84870e-01, 5.59750e-01],
         [1.99939e+01, 4.93034e+00, 1.16213e+01,  ..., 1.90718e-05, 6.45580e-01, 4.73927e-01],
         ...,
         [5.65543e+02, 6.05235e+02, 1.94246e+02,  ..., 5.69787e-04, 1.54198e-01, 7.98783e-01],
         [5.89485e+02, 6.02949e+02, 1.10033e+02,  ..., 1.18183e-03, 1.32517e-01, 8.48703e-01],
         [6.15738e+02, 6.08592e+02, 1.12142e+02,  ..., 1.70467e-04, 1.10823e-01, 8.83022e-01]]], grad_fn=<CatBackward0>), [tensor([[[[[ 1.39935e+00,  3.43303e-01, -7.15889e-02,  ..., -1.02724e+01,  2.49536e-01,  3.00444e-01],
           [ 1.04618e-01,  3.85729e-01,  3.97472e-01,  ..., -1.02269e+01,  3.42796e-01,  2.40149e-01],
           [-1.51645e-03,  2.33643e-01,  1.56360e-01,  ..., -1.08673e+01,  5.99668e-01, -1.04388e-01],
           ...,
           [ 2.97445e-01, -7.05611e-02,  6.55828e-02,  ..., -

In [257]:
iou = 0.45
score = 0.50

allowed_classes = [0,1]

# This stores the rider & motorcycle boxes, scores, classes and number of objects detected for each image in the testing set
all_boxes_R_M = []
all_scores_R_M = []
all_classes_R_M = []
all_num_objects_R_M = []

for rider_motor_bbox in all_rider_motor_bbox:
    preds = non_max_suppression(rider_motor_bbox,
                        score,
                        iou,
                        labels=(),
                        multi_label=True,
                        agnostic=False,
                        max_det=300)
    # print(preds)
    ind = torchvision.ops.nms(preds[0][:,:4], preds[0][:,4], 0.6)
    preds[0] = preds[0][ind]
    preds = preds[0]
    # print(preds)

    num_objects_R_M = preds.shape[0]
    boxes_R_M = preds[:,:4].detach().numpy()
    boxes_R_M = boxes_R_M[:] / input_size
    scores_R_M = preds[:,4].detach().numpy()
    scores_R_M = scores_R_M[:]
    classes_R_M = preds[:,5].detach().numpy()
    classes_R_M = classes_R_M[:]

    deleted_indx = []
    for i in range(num_objects_R_M):
        class_indx = int(classes_R_M[i])
        if class_indx not in allowed_classes:
            deleted_indx.append(i)
    boxes_R_M = np.delete(boxes_R_M, deleted_indx, axis=0)
    scores_R_M = np.delete(scores_R_M, deleted_indx, axis=0)
    classes_R_M = np.delete(classes_R_M, deleted_indx, axis=0)
    num_objects_R_M = len(classes_R_M)
    print("box ")
    print(boxes_R_M)
    print("scores ")
    print(scores_R_M)
    # MOTORCYCLE CLASS CHANGED TO 3 FROM 1
    classes_R_M[classes_R_M == 1] = 3

    all_boxes_R_M.append(boxes_R_M)
    all_scores_R_M.append(scores_R_M)
    all_classes_R_M.append(classes_R_M)
    all_num_objects_R_M.append(num_objects_R_M)

box 
[[    0.85979      0.6196     0.99388     0.72733]
 [      0.267     0.61212       0.436     0.80329]
 [    0.41241     0.56916     0.43212     0.59776]]
scores 
[    0.85407     0.72247     0.63022]
box 
[[    0.66857     0.60346     0.79763     0.92206]
 [    0.67672     0.54787     0.79725     0.83872]
 [    0.15111     0.69985     0.44151      1.0022]]
scores 
[    0.86323      0.8022     0.75367]
box 
[[    0.43939     0.66403     0.70432      1.0024]]
scores 
[    0.83804]
box 
[[   0.035037     0.43921     0.22491      1.0149]
 [    0.25322     0.45776      0.4218      1.0102]]
scores 
[    0.66975     0.64335]


In [258]:
import pandas as pd

# This stores the rider and motorcycle dataframes for each image in the test set
all_rider = []
all_motorcycle = []

# Getting the rider motorcycle dataframe 

for i, frame in enumerate(frames):
    # Bounding boxes are in normalized ymin, xmin, ymax, xmax
    original_h, original_w, _ = frame.shape
    classes_R_M = all_classes_R_M[i]
    boxes_R_M = all_boxes_R_M[i]

    #getting rider, motorcycle dataframe
    df = pd.DataFrame(classes_R_M, columns=['class_id'])
    ymin = boxes_R_M[:, 1]
    xmin = boxes_R_M[:, 0]
    ymax = boxes_R_M[:, 3]
    xmax = boxes_R_M[:, 2]
    df['x'] = pd.DataFrame(xmin + (xmax-xmin)/2, columns=['x'])
    df['y'] = pd.DataFrame(ymin + (ymax-ymin)/2, columns=['y'])
    df['w'] = pd.DataFrame(xmax-xmin, columns=['w'])
    df['h'] = pd.DataFrame(ymax-ymin, columns=['h'])
    rider = df.loc[df['class_id']==0]
    motorcycle = df.loc[df['class_id']==3]
    all_rider.append(rider)
    all_motorcycle.append(motorcycle)

### Assigning same Instance Ids to riders of a corresponding motorcycle

In [259]:
# assign instance id to each rider AND motorcycle
import sys
sys.path.append('../')
from core.association import motor_rider_iou, motor2_rider_iou

def get_instance(rider, motorcycle, iou_threshold):
    """
    args:
    rider, motorcycle : pd.DataFrame

    output:
    rider, motorycle : pd.DataFrame with a column named 'instance_id'
    """
    rider['instance_id'] = -1
    motorcycle['instance_id'] = -1
    
    for i in range(len(motorcycle)):
        motorcycle.iat[i,motorcycle.columns.get_loc('instance_id')] = i
        for j in range(len(rider)):
            if (motor_rider_iou(motorcycle.iloc[i], rider.iloc[j]) > iou_threshold):
                if (rider.iloc[j]['instance_id'] == -1):
                    rider.iat[j,rider.columns.get_loc('instance_id')] = i
                else:
                    instance = int(rider.iloc[j]['instance_id'])
                    instance_final = motor2_rider_iou(motorcycle.iloc[i], motorcycle.iloc[instance], rider.iloc[j], i, instance)
                    rider.iat[j,rider.columns.get_loc('instance_id')] = instance_final
    return rider, motorcycle

for i in range(len(all_rider)):
    all_rider[i], all_motorcycle[i] = get_instance(all_rider[i], all_motorcycle[i], 0.01)
    print("\n\nRider and Motorcycle Dataframe for image ", i)
    print(all_rider[i])
    print(all_motorcycle[i])



Rider and Motorcycle Dataframe for image  0
Empty DataFrame
Columns: [class_id, x, y, w, h, instance_id]
Index: []
   class_id         x         y         w         h  instance_id
0       3.0  0.926831  0.673466  0.134090  0.107730            0
1       3.0  0.351501  0.707707  0.169006  0.191167            1
2       3.0  0.422269  0.583457  0.019713  0.028598            2


Rider and Motorcycle Dataframe for image  1
   class_id         x         y         w         h  instance_id
1       0.0  0.736985  0.693293  0.120532  0.290846            0
   class_id         x         y         w         h  instance_id
0       3.0  0.733101  0.762762  0.129052  0.318598            0
2       3.0  0.296310  0.851022  0.290403  0.302343            1


Rider and Motorcycle Dataframe for image  2
Empty DataFrame
Columns: [class_id, x, y, w, h, instance_id]
Index: []
   class_id         x         y         w         h  instance_id
0       3.0  0.571854  0.833203  0.264934  0.338343            0


Rid

c:\Users\basur\AppData\Local\Programs\Python\Python310\lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


### Extracting ROIs from Images

In [275]:
def extract_roi(frame, rider, motorcycle) :
    """
    args:
    frame : np.array
    rider, motorcycle : pd.DataFrame

    output:
    roi_instances : list of np.array
    """
    roi_instances = []
    for i in range(len(motorcycle)):
        motor = motorcycle.loc[motorcycle['instance_id']==i]
        instance = motorcycle.iloc[i]['instance_id']
        ride = rider.loc[rider['instance_id']== instance]

        if (len(ride)==0):
            continue
        
        xmax = max(float(motor['x'] + motor['w']/2), max(ride['x'] + ride['w']/2))
        xmin = min(float(motor['x'] - motor['w']/2), min(ride['x'] - ride['w']/2))
        ymax = max(float(motor['y'] + motor['h']/2), max(ride['y'] + ride['h']/2))
        ymin = min(float(motor['y'] - motor['h']/2), min(ride['y'] - ride['h']/2))

        w = xmax - xmin
        h = ymax - ymin

        xmax = xmax + 0.05*w
        xmin = xmin - 0.05*w

        ymax = ymax + 0.05 * h
        ymin = ymin - 0.05 * h

        if (xmin < 0):
            xmin=0
        if (xmax >1):
            xmax=1
        if (ymax>1):
            ymax =1
        if(ymin<0):
            ymin =0

        t = int(ymin*frame.shape[0])
        l = int(xmin*frame.shape[1])
        b = int(ymax*frame.shape[0])
        r = int(xmax*frame.shape[1])

        if t<0 or l<0 or b<0 or r<0:
            continue
        roi_frame = frame[t:b, l:r]
        # roi_frame = frame
        original_position = (t,l,b,r)
        roi_dict = {'frame':roi_frame, 'original_position':original_position}
        print(roi_dict)
        roi_instances.append(roi_dict)

    return roi_instances

# Stores all rois in outputs/roi_images folder
all_roi_instances = []
for i in range(len(frames)):
    roi_instances = extract_roi(frames[i], all_rider[i], all_motorcycle[i])
    for j, roi_instance in enumerate(roi_instances):
        # convert to BGR
        roi_instance['frame'] = cv2.cvtColor(roi_instance['frame'], cv2.COLOR_RGB2BGR)
        cv2.imwrite("../outputs/outputs-images/yolov5/test_images/roi_images/roi_image_" + str(i) + "_" + str(j) + ".jpg", roi_instance['frame'])
        print("Roi", j, "of frame", i, "saved")
        roi_instance['frame'] = cv2.cvtColor(roi_instance['frame'], cv2.COLOR_BGR2RGB)
    all_roi_instances.append(roi_instances)

{'frame': array([[[102,  90,  66],
        [101,  89,  65],
        [ 99,  87,  63],
        ...,
        [ 41,  20,  25],
        [ 42,  23,  27],
        [ 39,  20,  24]],

       [[ 75,  64,  46],
        [ 72,  61,  43],
        [ 73,  62,  44],
        ...,
        [ 53,  30,  36],
        [ 53,  30,  36],
        [ 44,  21,  27]],

       [[ 30,  21,   4],
        [ 30,  21,   4],
        [ 32,  23,   6],
        ...,
        [ 61,  36,  42],
        [ 62,  37,  43],
        [ 45,  20,  26]],

       ...,

       [[ 78,  63,  56],
        [ 74,  59,  52],
        [ 75,  60,  53],
        ...,
        [ 69,  54,  51],
        [ 69,  54,  51],
        [ 71,  56,  53]],

       [[ 82,  67,  60],
        [ 74,  59,  52],
        [ 76,  61,  54],
        ...,
        [ 71,  56,  53],
        [ 73,  58,  55],
        [ 71,  56,  53]],

       [[ 74,  59,  52],
        [ 77,  62,  55],
        [ 78,  63,  56],
        ...,
        [ 72,  57,  54],
        [ 74,  59,  56],
        [ 69, 

## Helmet / No Helmet Detection (ROI Based)

In [274]:
# The following code loops through all the frames in the first loop. For each frame, via a series of nested loops, the helmet / no-helmet classifier is run on each ROI in the frame.
# The bounding boxes for each ROI are then extrapolated back to the original frame size and stored in a list. The same is done for the class and score of each ROI.

iou = 0.45
score = 0.50

# The following would be the output of ROI based detection for a single frame
all_frames_boxes_HNH = []
all_frames_classes_HNH = []
all_frames_scores_HNH = []
all_frames_num_objects_HNH = []
all_frames_ROI_sizes = []

for image_idx in range(len(all_roi_instances)):
    # roi instances for a single image
    roi_instances = all_roi_instances[image_idx]

    all_batch_data = []
    frame_size_ROI = []
    print(roi_instances)
    for i in range(len(roi_instances)):
        instance_frame = roi_instances[i]['frame']
        frame_size_ROI.append(instance_frame.shape)
        image_data = cv2.resize(frame, (input_size, input_size))
        image_data = image_data / 255.
        image_data = image_data[np.newaxis, ...].astype(np.float32)
        image_data = torch.from_numpy(image_data)
        image_data = image_data.permute((0,3,1,2))
        all_batch_data.append(image_data)

    # Getting helmet / no-helmet predictions
    all_final_HNH_boxes = []

    for i in range(len(all_batch_data)):
        batch_data = all_batch_data[i]
        H_NH_boxes = infer_RHNH(batch_data)
        all_final_HNH_boxes.append(H_NH_boxes)


    # NMS

    all_bboxes_HNH = []
    all_scores_HNH = []
    all_classes_HNH = []
    all_num_objects_HNH = []

    for i in range(len(all_final_HNH_boxes)):
        boxes = all_final_HNH_boxes[i]
        preds = non_max_suppression(boxes,
                        score,
                        iou,
                        labels=(),
                        multi_label=True,
                        agnostic=False,
                        max_det=300)
        # print(preds)
        ind = torchvision.ops.nms(preds[0][:,:4], preds[0][:,4], 0.6)
        preds[0] = preds[0][ind]
        preds = preds[0]
        # print(preds)

        num_objects_HNH = preds.shape[0]
        bboxes_HNH = preds[:,:4].detach().numpy()
        bboxes_HNH = bboxes_HNH[:] / input_size
        scores_HNH = preds[:,4].detach().numpy()
        scores_HNH = scores_HNH[:]
        classes_HNH = preds[:,5].detach().numpy()
        classes_HNH = classes_HNH[:]


        deleted_indx = []
        allowed_classes = [0, 1]
        for i in range(num_objects_HNH):
            class_indx = int(classes_HNH[i])
            if class_indx not in allowed_classes:
                deleted_indx.append(i)
        bboxes_HNH = np.delete(bboxes_HNH, deleted_indx, axis=0)
        scores_HNH = np.delete(scores_HNH, deleted_indx, axis=0)
        classes_HNH = np.delete(classes_HNH, deleted_indx, axis=0)
        num_objects_HNH = len(classes_HNH)

        classes_HNH[classes_HNH == 1] = 2
        classes_HNH[classes_HNH == 0] = 1

        all_bboxes_HNH.append(bboxes_HNH)
        all_scores_HNH.append(scores_HNH)
        all_classes_HNH.append(classes_HNH)
        all_num_objects_HNH.append(num_objects_HNH)

    # This will store the final bounding boxes, scores and classes for the image (Helmet / No Helmet)
    final_bboxes_HNH = []
    final_scores_HNH = []
    final_classes_HNH = []
    final_num_objects_HNH = 0

    # convert the bounding box to the original image in yolo format and store it in final_bboxes_HNH
    for i in range(len(roi_instances)):
        original_position = roi_instances[i]['original_position']
        full_frame_width = frames[image_idx].shape[1]
        full_frame_height = frames[image_idx].shape[0]
        for j in range(all_num_objects_HNH[i]):
            xmin = int(all_bboxes_HNH[i][j][0] * frame_size_ROI[i][1])
            ymin = int(all_bboxes_HNH[i][j][1] * frame_size_ROI[i][0])
            xmax = int(all_bboxes_HNH[i][j][2] * frame_size_ROI[i][1])
            ymax = int(all_bboxes_HNH[i][j][3] * frame_size_ROI[i][0])
            xmin = xmin + original_position[1]
            xmax = xmax + original_position[1]
            ymin = ymin + original_position[0]
            ymax = ymax + original_position[0]
            
            # convert to yolo format 
            x_center = (xmin + xmax) / (2 * full_frame_width)
            y_center = (ymin + ymax) / (2 * full_frame_height)
            width = (xmax - xmin) / full_frame_width
            height = (ymax - ymin) / full_frame_height
            final_bboxes_HNH.append([x_center, y_center, width, height])
            final_scores_HNH.append(all_scores_HNH[i][j])
            final_classes_HNH.append(all_classes_HNH[i][j])
            final_num_objects_HNH += 1

    all_frames_boxes_HNH.append(final_bboxes_HNH)
    all_frames_scores_HNH.append(final_scores_HNH)
    all_frames_classes_HNH.append(final_classes_HNH)
    all_frames_num_objects_HNH.append(final_num_objects_HNH)
    all_frames_ROI_sizes.append(frame_size_ROI)
    print("Done with image: ", image_idx)

print(all_frames_boxes_HNH)


[]
Done with image:  0
[{'frame': array([[[102,  90,  66],
        [101,  89,  65],
        [ 99,  87,  63],
        ...,
        [ 41,  20,  25],
        [ 42,  23,  27],
        [ 39,  20,  24]],

       [[ 75,  64,  46],
        [ 72,  61,  43],
        [ 73,  62,  44],
        ...,
        [ 53,  30,  36],
        [ 53,  30,  36],
        [ 44,  21,  27]],

       [[ 30,  21,   4],
        [ 30,  21,   4],
        [ 32,  23,   6],
        ...,
        [ 61,  36,  42],
        [ 62,  37,  43],
        [ 45,  20,  26]],

       ...,

       [[ 78,  63,  56],
        [ 74,  59,  52],
        [ 75,  60,  53],
        ...,
        [ 69,  54,  51],
        [ 69,  54,  51],
        [ 71,  56,  53]],

       [[ 82,  67,  60],
        [ 74,  59,  52],
        [ 76,  61,  54],
        ...,
        [ 71,  56,  53],
        [ 73,  58,  55],
        [ 71,  56,  53]],

       [[ 74,  59,  52],
        [ 77,  62,  55],
        [ 78,  63,  56],
        ...,
        [ 72,  57,  54],
        [ 74,  

### R+M predicted boxes annotated on Images stored in outpts folder

In [ ]:
# plot rider and motorcycle bounding boxes

for image_idx in range(len(frames2)):
    frame = frames2[image_idx]
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    original_h, original_w, _ = frames2[image_idx].shape
    rider = all_rider[image_idx]
    motorcycle = all_motorcycle[image_idx]
    for motor in motorcycle.iterrows():
        xmin = motor[1]['x'] - motor[1]['w']/2
        ymin = motor[1]['y'] - motor[1]['h']/2
        xmax = motor[1]['x'] + motor[1]['w']/2
        ymax = motor[1]['y'] + motor[1]['h']/2
        xmin = int(xmin*original_w)
        ymin = int(ymin*original_h)
        xmax = int(xmax*original_w)
        ymax = int(ymax*original_h)
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255,0,0), 5)
        cv2.putText(frame, 'motorcycle', (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (12,255,36), 5)

    for ride in rider.iterrows():
        xmin = ride[1]['x'] - ride[1]['w']/2
        ymin = ride[1]['y'] - ride[1]['h']/2
        xmax = ride[1]['x'] + ride[1]['w']/2
        ymax = ride[1]['y'] + ride[1]['h']/2
        xmin = int(xmin*original_w)
        ymin = int(ymin*original_h)
        xmax = int(xmax*original_w)
        ymax = int(ymax*original_h)
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255,255,0), 5)
        cv2.putText(frame, 'rider', (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (12,255,36), 5)     
    
    # save in folder 
    cv2.imwrite('../outputs/outputs-images/yolov5/test_images/annotated_R_M/' + str(image_idx) + '.jpg', frame)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # save annotations in txt file in yolo format
    with open('../outputs/outputs-images/yolov5/test_images/annotated_R_M/' + str(image_idx) + '.txt', 'w') as f:
        for motor in motorcycle.iterrows():
            f.write('3' + ' ' + str(motor[1]['x']) + ' ' + str(motor[1]['y']) + ' ' + str(motor[1]['w']) + ' ' + str(motor[1]['h']) + '\n')
        for ride in rider.iterrows():
            f.write('0' + ' ' + str(ride[1]['x']) + ' ' + str(ride[1]['y']) + ' ' + str(ride[1]['w']) + ' ' + str(ride[1]['h']) + '\n')

### Helmet / No Helmet Annotated Images stored in outputs folder

In [ ]:
# show the bounding box and the label for each image in outputs/annotated_HNH
for image_idx in range(len(all_frames_boxes_HNH)):
    frame = frames[image_idx]
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    final_bboxes_HNH = all_frames_boxes_HNH[image_idx]
    final_scores_HNH = all_frames_scores_HNH[image_idx]
    final_classes_HNH = all_frames_classes_HNH[image_idx]
    for i in range(all_frames_num_objects_HNH[image_idx]):
        xmin = int((final_bboxes_HNH[i][0] - final_bboxes_HNH[i][2] / 2) * frame.shape[1])
        ymin = int((final_bboxes_HNH[i][1] - final_bboxes_HNH[i][3] / 2) * frame.shape[0])
        xmax = int((final_bboxes_HNH[i][0] + final_bboxes_HNH[i][2] / 2) * frame.shape[1])
        ymax = int((final_bboxes_HNH[i][1] + final_bboxes_HNH[i][3] / 2) * frame.shape[0])
        if final_classes_HNH[i] == 1:
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 7)
        else :
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 7)
        cv2.putText(frame, '{} {:.2f}'.format(int(final_classes_HNH[i]), final_scores_HNH[i]),
                    (xmin, ymin - 5), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 7)
    cv2.imwrite('../outputs/outputs-images/yolov5/test_images/annotated_HNH/' + str(image_idx) + '.jpg', frame)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    print("Done with image: ", image_idx)
    # save annotations in txt file in yolo format
    with open('../outputs/outputs-images/yolov5/test_images/annotated_HNH/' + str(image_idx) + '.txt', 'w') as f:
        for i in range(all_frames_num_objects_HNH[image_idx]):
            f.write(str(int(final_classes_HNH[i])) + ' ' + str(final_bboxes_HNH[i][0]) + ' ' + str(final_bboxes_HNH[i][1]) + ' ' + str(final_bboxes_HNH[i][2]) + ' ' + str(final_bboxes_HNH[i][3]) + '\n')

Done with image:  0
Done with image:  1
Done with image:  2
Done with image:  3


### Getting Test Ground Truth Annotations

In [ ]:
all_frames_classes_HNH[i] = [int(j) for j in all_frames_classes_HNH[i]]

In [ ]:
# Get the ground truth annotations from txt files in data/testing_data_310Images/final_test_set/

# Contains the class id (0/1) and bounding box coordinates of all instances in all frames
gt_boxes = []
gt_classes = []

    
for file in os.listdir('../data/testing_data_310Images/final_test_set/'):
    if file.endswith(".txt"):
        file_name = file.split('.')[0]     
        f = open('../data/testing_data_310Images/final_test_set/'+file, 'r')
        lines = f.readlines()
        gt_box = []
        gt_class = []
        for line in lines:
            line = line.split()
            class_id = int(line[0]) + 1 # change from 0/1 to 1/2
            boxes = [float(line[1]), float(line[2]),
                        float(line[3]), float(line[4])]
            gt_box.append(boxes)
            gt_class.append(class_id)
        gt_boxes.append(gt_box)
        gt_classes.append(gt_class)
        f.close()

gt_boxes = gt_boxes[0:4]
gt_classes = gt_classes[0:4]

print("Number of frames with ground truth annotations: ", len(gt_boxes))

print(gt_boxes)
print(gt_classes)

Number of frames with ground truth annotations:  4
[[[0.360758, 0.583181, 0.022661, 0.047843], [0.405401, 0.575676, 0.02351, 0.048852], [0.381323, 0.573157, 0.022375, 0.049852]], [[0.210117, 0.634699, 0.064849, 0.127824], [0.74407, 0.573292, 0.037359, 0.062657], [0.311271, 0.606505, 0.051458, 0.119046], [0.265453, 0.624046, 0.052865, 0.116537]], [[0.589701, 0.580815, 0.050047, 0.097741], [0.530844, 0.60212, 0.049344, 0.102759], [0.479034, 0.607755, 0.05287, 0.104009]], [[0.143862, 0.551366, 0.114193, 0.231824], [0.358497, 0.573921, 0.092339, 0.211787], [0.244659, 0.542593, 0.090224, 0.219296]]]
[[2, 2, 2], [2, 2, 2, 2], [2, 2, 2], [2, 2, 2]]


### Metric Calculation

In [ ]:
from core.association import iou

def helmets_iou(h1, h2):
  x, y, w, h = h1[0], h1[1], h1[2], h1[3]
  h1_pascal = [[x-w/2, y-h/2], [x+w/2, y-h/2], [x+w/2, y+h/2], [x-w/2, y+h/2]]
  x, y, w, h = h2[0], h2[1], h2[2], h2[3]
  h2_pascal = [[x-w/2, y-h/2], [x+w/2, y-h/2], [x+w/2, y+h/2], [x-w/2, y+h/2]]
  return iou(h1_pascal, h2_pascal)

def iou_mat(gt, pred):
    '''
    returns the iou mat of shape (gt.shape[0], pred.shape[0])
    '''
    mat = np.zeros((len(gt), len(pred)))
    for i in range(len(gt)):
        for j in range(len(pred)):
            mat[i,j] = helmets_iou(gt[i], pred[j])
    return mat

predicted_boxes = all_frames_boxes_HNH
predicted_classes = all_frames_classes_HNH

total_predicted = 0
total_gt = 0
fp = 0
fn = 0
tp = 0
iou_thresh = 0.5
for i in range(len(gt_boxes)):
    # if(i!=3):
    #     continue
    gt_box = gt_boxes[i]
    gt_class = gt_classes[i]
    pred_box = predicted_boxes[i]
    total_gt += len(gt_box)
    total_predicted += len(pred_box)
    pred_class = predicted_classes[i]
    mat = iou_mat(gt_box, pred_box)
    ind = np.argsort(mat, axis = -1)
    sorted_mat = np.sort(mat, axis = -1)

    gt_assign = np.ones((len(gt_box))) * -1
    gt_assign_class = np.ones((len(gt_box))) * -1

    for i in range(len(gt_box)):
        if(sorted_mat.shape[1] > 0):
            if(sorted_mat[i,-1] > iou_thresh and pred_class[ind[i, -1]] == gt_class[i]):
                gt_assign[i] = ind[i, -1]

    val = gt_assign[gt_assign != -1]

    if (len(np.unique(val)) != len(val)):
        print("yoyo")
    else:
        fn += len(gt_assign[gt_assign == -1])
        fp += len(pred_box) - len(val)
        tp += len(val)

print("Total Predicted: ", total_predicted)
print("Total Ground Truth: ", total_gt)
print("True Positives: ", tp)
print("False Positives: ", fp)
print("False Negatives: ", fn)
print("Precision: ", tp/(tp+fp))
print("Recall: ", tp/(tp+fn))
print("f1-score: ", 2*tp/(2*tp+fp+fn))

Total Predicted:  0
Total Ground Truth:  13
True Positives:  0
False Positives:  0
False Negatives:  13


ZeroDivisionError: division by zero